# 第八章：类与对象

本章主要关注点的是和类定义有关的常见编程模型。包括让对象支持常见的Python特性、特殊方法的使用、类封装技术、继承、内存管理以及有用的设计模式。

## 8.1 改变对象的字符串显示

* 问题

改变对象实例的打印或显示输出，让它们更具可读性。

* 解决方案

要改变一个实例的字符串表示，可重新定义它的 `__str__()` 和 `__repr__()` 方法。

In [2]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)
    
    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)

`__repr__()` 返回一个实例的代码表示形式，通常用来构造这个实例。内置的 `repr()` 函数返回这个字符串，跟我们使用交互式解释器显示的值是一样的。

`__str__()` 将实例转换为一个字符串，使用 `str()` 或 `print()` 函数会输出这个字符串。

In [3]:
p = Pair(3, 4)
p  # __repr__() output

Pair(3, 4)

In [5]:
print(p)  # __str__() output

(3, 4)


上述代码还演示了再格式化的时候怎样使用不同的字符串表现形式。

特别来讲，`!r` 格式化代码指明输出使用 `__repr__()` 来代替默认的 `__str()__`。

可以使用Pair类测试

In [6]:
p = Pair(3, 4)
print('p is {0!r}'.format(p))

p is Pair(3, 4)


In [7]:
print('p is {0}'.format(p))

p is (3, 4)


* 讨论

自定义 `__repr__()` 和 `__str()__` 可以简化调试和实例输出。我们只用通过打印实例，程序员就会看到实例更加详细的信息

`__repr__()` 生成的文本字符串标准做法是需要让 `eval(repr(x)) == x` 为真。


如果不行或者不希望有这种行为，那么通常就让它产生一段有帮助意义的文本，并且以 `<` 和 `>` 括起来

In [ ]:
f = open('file.dat')

如果 `__str__()` 没有被定义，那么就会使用 `__repr__()` 来代替输出。

上面的 `format()` 格式化方法 `{0.x}` 对应的是第1个参数的x属性。因此，在下面的函数中，0实际上指的就是 self 本身：
```python
def __repr__(self):
    return 'Pair({0.x!r}, {0.y!r})'.format(self)
```

作为这种实现的一个替代，你也可以使用 `%` 操作符，就像下面这样：
```python
def __repr__(self):
    return 'Pair(%r, %r)' % (self.x, self.y)
```

## 8.2 自定义字符串的格式化

* 问题

通过 `format()` 函数和字符串方法使得一个对象能支持自定义的格式化。

* 解决方案

为了自定义字符串的格式化，我们需要在类上面定义 `__format__()` 方法。

In [8]:
_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}/{d.day}/{d.year}',
    'dmy' : '{d.day}/{d.month}/{d.year}'
    }

In [14]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        print(self)
        return fmt.format(d=self)

In [15]:
# 现在 `Date` 类的实例可以支持格式化操作了
d = Date(2012, 12, 21)
format(d)

'2012-12-21'

In [16]:
format(d, 'mdy')

'12/21/2012'

In [12]:
'The date is {:ymd}'.format(d)

'The date is 2012-12-21'

In [13]:
'The date is {:mdy}'.format(d)

'The date is 12/21/2012'

* 讨论

`__format__()` 方法给Python的字符串格式化功能提供了一个钩子。

这里需要着重强调的是格式化代码的解析工作完全由类自己决定。因此，格式化代码可以是任何值。

例如，参考下面来自 datetime 模块中的代码：

In [17]:
from datetime import date

d = date(2012, 12, 21)
format(d)

'2012-12-21'

In [18]:
format(d, '%A, %B%d, %Y')

'Friday, December21, 2012'

In [19]:
'The end is {:%d %b %Y}. Goodbye'.format(d)

'The end is 21 Dec 2012. Goodbye'

对于内置类型的格式化有一些标准的约定。 可以参考 [string模块文档](https://docs.python.org/3/library/string.html) 说明。

## 8.3 让对象支持上下文管理协议

* 问题

让对象支持上下文管理协议(with语句)。

* 解决方案

为了让一个对象兼容 `with` 语句，你需要实现 `__enter__()` 和 `__exit__()` 方法。

In [20]:
# 假设创建一个能够创建网络连接的类：
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.sock = None

    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

这个类的核心功能就是表示一条网络连接，但是初始化的时候并不会做任何事情（比如它并没有建立一个连接）。连接的建立和关闭是使用 `with` 语句自动完成的。

In [21]:
from functools import partial

conn = LazyConnection(('www.python.org', 80))
# connection closed
with conn as s:
    # conn.__enter__() executes: connection open
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))
    # conn.__exit__() executes: connection closed

* 讨论

编写上下文管理器的主要原理是你的代码会放到 `with` 语句块中执行。
1. 当出现 with 语句的时候，对象的 `__enter__()` 方法被触发，它返回的值(如果有的话)会被赋值给 as 声明的变量。
2. 然后，with 语句块里面的代码开始执行。
3. 最后，`__exit__()` 方法被触发进行清理工作。


不管 with 代码块中发生什么，上面的控制流都会执行完，就算代码块中发生了异常也是一样的。

事实上，`__exit__()` 方法的三个参数包含了异常类型、异常值和追溯信息(如果有的话)。

`__exit__()` 方法能自己决定怎样利用这个异常信息，或者忽略它并返回一个None值。

如果 `__exit__()` 返回 `True` ，那么异常会被清空，就好像什么都没发生一样， with 语句后面的程序继续在正常执行。

还有一个细节问题就是 `LazyConnection` 类是否允许多个 `with` 语句来嵌套使用连接。很显然，上面的代码定义中一次只能允许一个 `socket` 连接，如果正在使用一个 `socket` 的时候又重复使用 `with` 语句，就会产生一个异常了。

不过可以修改下上面的实现来解决这个问题：

In [22]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.connections = []

    def __enter__(self):
        sock = socket(self.family, self.type)
        sock.connect(self.address)
        self.connections.append(sock)
        return sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.connections.pop().close()

# Example use
from functools import partial

conn = LazyConnection(('www.python.org', 80))
with conn as s1:
    pass
    with conn as s2:
        pass
        # s1 and s2 are independent sockets

在第二个版本中，`LazyConnection` 类可以被看做是某个连接工厂。在内部，一个列表被用来构造一个栈。
* 每次 `__enter__()` 方法执行的时候，它复制创建一个新的连接并将其加入到栈里面。
* `__exit__()` 方法简单的从栈中弹出最后一个连接并关闭它。

在需要管理一些资源比如文件、网络连接和锁的编程环境中，使用上下文管理器是很普遍的。这些资源的一个主要特征是它们必须**被手动的关闭或释放来确保程序的正确运行**。

例如，如果你请求了一个锁，那么你必须确保之后释放了它，否则就可能产生死锁。

通过实现 `__enter__()` 和 `__exit__()` 方法并使用 with 语句可以很容易的避免这些问题， 因为 __exit__() 方法可以让你无需担心这些了。

在 `contextmanager` 模块中有一个标准的上下文管理方案模板，可参考9.22小节。 同时在12.6小节中还有一个对本节示例程序的线程安全的修改版。

## 8.4 创建大量对象时节省内存方法
* 问题

程序要创建大量(可能上百万)的对象，导致占用很大的内存。

* 解决方案
对于主要是用来当成简单的数据结构的类而言，你可以通过给类添加 `__slots__` 属性来极大的减少实例所占的内存。

In [24]:
class Date:
    __slots = ['year', 'month', 'day']
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

当你定义 `__slots__` 后，Python就会为实例使用一种更加紧凑的内部表示。

实例通过一个很小的固定大小的数组来构建，而不是为每个实例定义一个字典，这跟元组或列表很类似。

在 `__slots__` 中列出的属性名在内部被映射到这个数组的特定索引上。

使用 `slots` 一个不好的地方就是**不能再给实例添加新的属性**了，只能使用在 `__slots__` 中定义的那些属性名。

* 讨论

使用 `slots` 后节省的内存会跟存储属性的数量和类型有关。不过，一般来讲，使用到的内存总量和将数据存储在一个元组中差不多。

为了给你一个直观认识，假设你不使用slots直接存储一个Date实例， 在64位的Python上面要占用428字节，而如果使用了slots，内存占用下降到156字节。
如果程序中需要同时创建大量的日期实例，那么这个就能极大的减小内存使用量了。

尽管slots看上去是一个很有用的特性，很多时候你还是得减少对它的使用冲动。
Python的很多特性都依赖于普通的基于字典的实现。

另外，定义了slots后的类不再支持一些普通类特性了，比如多继承。
大多数情况下，你应该只在那些经常被使用到的用作数据结构的类上定义slots (比如在程序中需要创建某个类的几百万个实例对象)。

关于 `__slots__` 的一个常见误区是它可以作为一个封装工具来防止用户给实例增加新的属性。
尽管使用slots可以达到这样的目的，但是这个并不是它的初衷。 __slots__ 更多的是用来作为一个内存优化工具。